In [13]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [2]:
spark_session = SparkSession.builder.appName("groupByApply").getOrCreate()
spark_session

In [3]:
df = spark_session.read.csv("test3.csv", header=True, inferSchema=False)
df = df.withColumn("salary", df.salary.cast("int"))
df.show()

+---------+------------+------+
|     Name| Departments|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



In [4]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- salary: integer (nullable = true)



In [5]:
df.groupBy("Departments").sum().show()

+------------+-----------+
| Departments|sum(salary)|
+------------+-----------+
|         IOT|      15000|
|    Big Data|      15000|
|Data Science|      43000|
+------------+-----------+



In [6]:
df.groupBy("Departments").count().show()

+------------+-----+
| Departments|count|
+------------+-----+
|         IOT|    2|
|    Big Data|    4|
|Data Science|    4|
+------------+-----+



In [7]:
test1_df = spark_session.read.csv("test2.csv", header=True, inferSchema = True)
test1_df = test1_df.na.drop()
test1_df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [9]:
test1_df.show() 

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [24]:
feature_assembler = VectorAssembler(inputCols = ["age", "Experience"], outputCol="Independent_Features")
output = feature_assembler.transform(test1_df)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent_Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [11]:
finalized_data = output.select("Independent_Features", "Salary")
finalized_data.show()

+--------------------+------+
|Independent_Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [20]:
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol="Independent_Features", labelCol="Salary")
regressor=regressor.fit(train_data)

In [21]:
regressor.coefficients

DenseVector([-258.8832, 1670.0508])

In [ ]:
pred_results = regressor.evaluate(train_data)
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent_Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|17928.934010152232|
|          [29.0,4.0]| 20000| 19715.73604060919|
|          [30.0,8.0]| 25000|26137.055837563454|
|         [31.0,10.0]| 30000|29218.274111675117|
+--------------------+------+------------------+

